<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/crim-ca/geoimagenet/blob/master/seg_model_packaging.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/crim-ca/geoimagenet/blob/master/seg_model_packaging.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/crim-ca/geoimagenet/blob/master/seg_model_packaging.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# SegNet Model Packaging for GeoImageNet

This notebook explains how to package a SegNet model using the thelper library. More details about thelper export function can be found here:

https://thelper.readthedocs.io/en/latest/user-guide.html#export-model

First, we need to install the required libraries.

### Environnement Configuration

In [8]:
%%bash
pip3 --quiet install  torch torchvision pillow gitpython lz4 matplotlib numpy pyyaml scikit-learn six tqdm h5py opencv-python  pretrainedmodels albumentations pyyaml
pip3 --quiet install affine geojson shapely pyproj hdf5plugin
pip uninstall thelper
rm -rf ./thelper
git clone https://github.com/plstcharles/thelper
pip3 install --quiet -e ./thelper

Cloning into 'thelper'...


We also install an external repo containing new models as well as useful functions.

In [9]:
!git clone https://github.com/crim-ca/gin-model-repo
!pip3 install --quiet -e ./gin-model-repo

Cloning into 'gin-model-repo'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 35 (delta 9), reused 31 (delta 8), pack-reused 0
Unpacking objects: 100% (35/35), done.


The first time you are installing thelper, you may need to restart the current session

In [ ]:
import os
os.kill(os.getpid(), 9)

Make sure thelper is imported correctly

In [1]:
import os
import torch
import torchvision
import thelper
import ginmodelrepo

thelper.utils.init_logger()

We download a UNet trained on DeepGlobe:

In [2]:
from ginmodelrepo.util import maybe_download_and_extract
maybe_download_and_extract('1ZHWFysFvIrdBELQKfrNXk7SwUSegqTaY','/content/model.pth')

Download finished. Extracting files.
Done.


Name your model

In [3]:
model_name = 'deepglobe-unet'

### Configuration

Specify your class names:

In [4]:
class_names = ["Background", "AgriculturalLand", "BarrenLand", "ForestLand", "RangeLand", "UrbanLand", "Water"]

#### Consulting the Taxonomy
You can consult the GeoImageNet Taxonomies, at level 0 it will you give how many taxonomies are available:

In [5]:
import requests
import json
response = requests.get('https://geoimagenet.ca/api/v1/taxonomy_classes?depth=0')
taxonomy_classes = json.loads(response.content)
print(taxonomy_classes)

[{'id': 205, 'name_fr': 'Couverture de sol', 'taxonomy_id': 2, 'code': 'COUV', 'name_en': 'Land cover', 'children': []}, {'id': 1, 'name_fr': 'Objets', 'taxonomy_id': 1, 'code': 'OBJE', 'name_en': 'Objects', 'children': []}]


You can see that two taxonomies are available: Land cover and Objects. We can consult the Land Cover taxonomy at level 1

In [6]:
response = requests.get('https://geoimagenet.ca/api/v1/taxonomy_classes?taxonomy_name=land-cover&depth=1')
taxonomy_classes = json.loads(response.content)
print(taxonomy_classes)

[{'id': 205, 'name_fr': 'Couverture de sol', 'taxonomy_id': 2, 'code': 'COUV', 'name_en': 'Land cover', 'children': [{'id': 206, 'name_fr': 'Urbain ou bâti', 'taxonomy_id': 2, 'code': 'URBA', 'name_en': 'Urban or Built-up Land', 'children': [{'id': 207, 'name_fr': 'Zone résidentielle', 'taxonomy_id': 2, 'code': 'RESD', 'name_en': 'Residential', 'children': [{'id': 208, 'name_fr': 'Densité faible', 'taxonomy_id': 2, 'code': 'DENS', 'name_en': 'Low density', 'children': []}, {'id': 209, 'name_fr': 'Densité moyenne', 'taxonomy_id': 2, 'code': 'DENI', 'name_en': 'Medium density', 'children': []}, {'id': 210, 'name_fr': 'Densité élevée', 'taxonomy_id': 2, 'code': 'DENT', 'name_en': 'High density', 'children': []}, {'id': 211, 'name_fr': 'Parc de maisons mobiles', 'taxonomy_id': 2, 'code': 'PARC', 'name_en': 'Mobile home park', 'children': []}]}, {'id': 212, 'name_fr': 'Zone commerciales et services', 'taxonomy_id': 2, 'code': 'COMC', 'name_en': 'Commercial and Services', 'children': [{'id':

Check that your classes are defined in the taxonomy

In [7]:

response = requests.get('https://geoimagenet.ca/api/v1/taxonomy_classes?taxonomy_name=land-cover')
taxonomy_classes = json.loads(response.content)
print(taxonomy_classes)

[{'id': 205, 'name_fr': 'Couverture de sol', 'taxonomy_id': 2, 'code': 'COUV', 'name_en': 'Land cover', 'children': [{'id': 206, 'name_fr': 'Urbain ou bâti', 'taxonomy_id': 2, 'code': 'URBA', 'name_en': 'Urban or Built-up Land', 'children': [{'id': 207, 'name_fr': 'Zone résidentielle', 'taxonomy_id': 2, 'code': 'RESD', 'name_en': 'Residential', 'children': [{'id': 208, 'name_fr': 'Densité faible', 'taxonomy_id': 2, 'code': 'DENS', 'name_en': 'Low density', 'children': []}, {'id': 209, 'name_fr': 'Densité moyenne', 'taxonomy_id': 2, 'code': 'DENI', 'name_en': 'Medium density', 'children': []}, {'id': 210, 'name_fr': 'Densité élevée', 'taxonomy_id': 2, 'code': 'DENT', 'name_en': 'High density', 'children': []}, {'id': 211, 'name_fr': 'Parc de maisons mobiles', 'taxonomy_id': 2, 'code': 'PARC', 'name_en': 'Mobile home park', 'children': []}]}, {'id': 212, 'name_fr': 'Zone commerciales et services', 'taxonomy_id': 2, 'code': 'COMC', 'name_en': 'Commercial and Services', 'children': [{'id':

Define a class mapping between your class names and the taxonomy ids, add 999 for the background class

In [8]:
class_mapping = [('Background', '999'), ('AgriculturalLand', '223'), ('BarrenLand', '252'), ('ForestLand', '233'), ('RangeLand', '229'), ('UrbanLand', '201'), ('Water', '239')]

#### Model Configuration

Define the task as a Segmentation task:

In [9]:
task_config = {
    "type": "thelper.tasks.Segmentation",
    "params": {
        "class_names": list(dict(class_mapping).keys()),
        "input_key": "image",
        'dontcare' : 255,
        "label_map_key": "label"
    }
}
print(task_config)

{'type': 'thelper.tasks.Segmentation', 'params': {'class_names': ['Background', 'AgriculturalLand', 'BarrenLand', 'ForestLand', 'RangeLand', 'UrbanLand', 'Water'], 'input_key': 'image', 'dontcare': 255, 'label_map_key': 'label'}}


We need to modify the task to include your class mapping

In [10]:
from ginmodelrepo.util import update_class_mapping
task = update_class_mapping(class_mapping, task_config)
print(task)

thelper.tasks.segm.Segmentation(class_names={999: 0, 223: 1, 252: 2, 233: 3, 229: 4, 201: 5, 239: 6}, input_key='image', label_map_key='label', meta_keys=[], dontcare=255, color_map={})


Define the data loader, you'll need to specify the bands you are using (starting with channel no 1):



In [11]:
channels = [1,2,3]
datasets_config = {
    "deepglobe_test": {
        "type": "thelper.data.geo.parsers.ImageFolderGDataset",
        "params": {"root": "/content/",
                    "image_key": "image",
                    "channels": channels

                    },
        "task": task
    }
}

Define the pre-processing pipeline as following:

In [12]:
loaders_config = {

    "base_transforms": [
        # this is necessary because the training used PIL which output images in a BGR order
        {
            "operation": "thelper.transforms.ToNumpy",
            "params": {"reorder_bgr": False},
        },
        {
            "operation": "thelper.transforms.CenterCrop",
            "params": {"size": 128},
        },
        {
            "operation": "thelper.transforms.NormalizeMinMax",
            "params": {"min": 0.0, "max": 255.0},
        },
        {
            "operation": "thelper.transforms.NormalizeZeroMeanUnitVar",
            "params": {
                "mean": [0.485, 0.456, 0.406],
                "std": [0.229, 0.224, 0.225]
            },
        },
        {
            "operation": "thelper.transforms.Transpose",
            "params": {
                "axes": [2,0,1]
            },
        },
    ],

}

Define the model configuration:
* type: your model specific class
* params: the parameters needed by that class
* state_dict: points to the model parameters



In [13]:
model_config = {
    "type": ginmodelrepo.EncoderDecoderNet,
    "params": {"num_classes": 7, "enc_type": 'resnet18', "dec_type": 'unet_scse', "num_filters": 16,
                "pretrained": True},
    'state_dict': '/content/model.pth',
    "task": task

}

### Export the model

Define the thelper export parameters, make sure the trace_input corresponds to the input layer

In [14]:
export_config = {
    "ckpt_name": model_name + ".pth",
    "trace_name": model_name + ".zip",
    "save_raw": True,
    "trace_input": "torch.rand(1, 3, 128, 128)",
    "task": task
}

Launch the export

In [15]:
# config = {"name": model_name, "model": model_config, "datasets": datasets_config, "loaders": loaders_config, "export": export_config}
config = {"name": model_name, "model": model_config, "datasets": datasets_config, "loaders": loaders_config,  "trainer": {}}
thelper.utils.bypass_queries = True     # avoid blocking ui query
thelper.cli.export_model(config, '/content/thelper-export')

[2020-09-07 14:33:33,520 - thelper.cli.export_model] INFO : exporting model 'deepglobe-unet'...
[2020-09-07 14:33:33,522 - thelper.utils.get_save_dir] INFO : output root directory = /content/thelper-export
[2020-09-07 14:33:33,774 - thelper.utils.get_save_dir] INFO : output session directory = /content/thelper-export/deepglobe-unet
[2020-09-07 14:33:33,776 - thelper.utils.get_save_dir] INFO : output logs directory = /content/thelper-export/deepglobe-unet/logs
[2020-09-07 14:33:33,778 - thelper.cli.export_model] DEBUG : exported checkpoint will be saved at '/content/thelper-export/deepglobe-unet'
[2020-09-07 14:33:33,780 - thelper.nn.utils] DEBUG : loading model
[2020-09-07 14:33:33,781 - thelper.nn.utils] DEBUG : loading model type/params current config
[2020-09-07 14:33:43,964 - thelper.nn.utils] DEBUG : model_type = <class 'ginmodelrepo.net.EncoderDecoderNet'>
[2020-09-07 14:33:43,965 - thelper.nn.utils] DEBUG : model_params = {'num_classes': 7, 'enc_type': 'resnet18', 'dec_type': 'u

[2020-09-07 14:33:44,744 - thelper.nn.utils] DEBUG : loading state dictionary from checkpoint into model
[2020-09-07 14:33:44,799 - thelper.nn.utils] DEBUG : previous model task = thelper.tasks.segm.Segmentation(class_names={999: 0, 223: 1, 252: 2, 233: 3, 229: 4, 201: 5, 239: 6}, input_key='image', label_map_key='label', meta_keys=[], dontcare=255, color_map={})
[2020-09-07 14:33:44,800 - thelper.nn.utils] DEBUG : refreshing model for new task = thelper.tasks.segm.Segmentation(class_names={999: 0, 223: 1, 252: 2, 233: 3, 229: 4, 201: 5, 239: 6}, input_key='image', label_map_key='label', meta_keys=[], dontcare=255, color_map={})
[2020-09-07 14:33:44,812 - thelper.nn.utils] INFO : module 'ginmodelrepo.net.EncoderDecoderNet' parameter count: 42814003
[2020-09-07 14:33:44,813 - thelper.nn.utils] INFO : EncoderDecoderNet(
  (encoder1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=T

We can check that the export can be reloaded

In [16]:
from io import BytesIO
model_file = '/content/thelper-export/deepglobe-unet/deepglobe-unet.pth'
with open(model_file, 'rb') as f:
  model_buffer = BytesIO(f.read())

model_checkpoint_info = thelper.utils.load_checkpoint(model_buffer)

[2020-09-07 14:34:02,796 - thelper.utils] DEBUG : testing availability of cuda device #0 (Tesla T4)


Testing the TorchScript version of your model

In [5]:
model = torch.jit.load('/content/thelper-export/deepglobe-unet/deepglobe-unet.zip')
model.eval()
print(model)
# print example output (should be same as during save)
x = torch.ones(1, 3, 128, 128)
print(model(x))

RecursiveScriptModule(
  original_name=ExternalModule
  (model): RecursiveScriptModule(
    original_name=EncoderDecoderNet
    (encoder1): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=Conv2d)
      (1): RecursiveScriptModule(original_name=BatchNorm2d)
      (2): RecursiveScriptModule(original_name=ReLU)
      (3): RecursiveScriptModule(original_name=MaxPool2d)
    )
    (encoder2): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=BasicBlock
        (conv1): RecursiveScriptModule(original_name=Conv2d)
        (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
        (relu): RecursiveScriptModule(original_name=ReLU)
        (conv2): RecursiveScriptModule(original_name=Conv2d)
        (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      )
      (1): RecursiveScriptModule(
        original_name=BasicBlock
        (conv1): RecursiveScriptModule(original_name=C

### Push your Model using GIN API

In [ ]:
To be finished...

In [17]:
model_path = '/content/thelper-export/deepglobe-unet/deepglobe-unet.pth'

In [18]:
!curl -X POST "https://geoimagenet.ca/ml/models" -H "accept: application/json" -H "Content-Type: application/json" -d "{ \"model_name\": \"{model_name}\", \"model_path\": \"{model_path}\"}"

<?xml version="1.0" encoding="utf-8"?>
<ExceptionReport version="1.0.0"
    xmlns="http://www.opengis.net/ows/1.1"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.opengis.net/ows/1.1 http://schemas.opengis.net/ows/1.1.0/owsExceptionReport.xsd">
    <Exception exceptionCode="NoApplicableCode" locator="AccessForbidden">
        <ExceptionText>Not authorized to access this resource. User does not meet required permissions.</ExceptionText>
    </Exception>
</ExceptionReport>